In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import warnings

# Desativa todos os avisos
warnings.filterwarnings("ignore")
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3070


In [2]:
# Definir hiperparâmetros
num_epochs = 20
batch_size = 64
learning_rate = 0.001
data_dir = './images/'  # Substitua pelo caminho para suas pastas de imagens
model_save_path = 'resnet50_model.pth'

In [3]:
# Transformações de dados
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Carregar os dados
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Verificar se há GPU disponível
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Treino usando {device}')

# Carregar o modelo ResNet50 pré-treinado
model = models.resnet50(pretrained=True)

# Modificar a última camada para corresponder ao número de classes no seu conjunto de dados
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# Função para treinar o modelo
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        # Cada época tem uma fase de treino e validação
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Modo de treino
            else:
                model.eval()  # Modo de avaliação

            running_loss = 0.0
            running_corrects = 0

            # Iterar sobre os dados
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zerar os gradientes do otimizador
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + otimização somente na fase de treino
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estatísticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    # Salvar o modelo treinado
    torch.save(model.state_dict(), model_save_path)
    print('Modelo salvo em:', model_save_path)

    return model

# Treinar o modelo
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)

Treino usando cuda:0
Epoch 0/19
----------
train Loss: 4.4718 Acc: 0.1134
val Loss: 3.7306 Acc: 0.3508
Epoch 1/19
----------
train Loss: 3.3670 Acc: 0.3929
val Loss: 2.4840 Acc: 0.5608
Epoch 2/19
----------
train Loss: 2.4860 Acc: 0.5155
val Loss: 1.7604 Acc: 0.6476
Epoch 3/19
----------
train Loss: 1.9943 Acc: 0.5892
val Loss: 1.4074 Acc: 0.7056
Epoch 4/19
----------
train Loss: 1.6683 Acc: 0.6332
val Loss: 1.1277 Acc: 0.7525
Epoch 5/19
----------
train Loss: 1.4627 Acc: 0.6703
val Loss: 0.9526 Acc: 0.7817
Epoch 6/19
----------
train Loss: 1.3116 Acc: 0.7004
val Loss: 0.8362 Acc: 0.8003
Epoch 7/19
----------
train Loss: 1.1987 Acc: 0.7149
val Loss: 0.7231 Acc: 0.8319
Epoch 8/19
----------
train Loss: 1.0911 Acc: 0.7490
val Loss: 0.6500 Acc: 0.8581
Epoch 9/19
----------
train Loss: 1.0040 Acc: 0.7606
val Loss: 0.5676 Acc: 0.8678
Epoch 10/19
----------
train Loss: 0.9218 Acc: 0.7797
val Loss: 0.4913 Acc: 0.8881
Epoch 11/19
----------
train Loss: 0.8723 Acc: 0.7837
val Loss: 0.4586 Acc: 

In [16]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import requests
from io import BytesIO


def get_class_names():
    class_names = [d.name for d in os.scandir('./images/train') if d.is_dir()]
    return class_names
class_names = get_class_names()
# Caminho para o modelo salvo
model_path = 'resnet50_model.pth'

# Carregar o modelo
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model.load_state_dict(torch.load(model_path))
model.eval()

# Função para processar a imagem
def process_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = preprocess(image).unsqueeze(0)
    return image

# Função para fazer a predição
def predict(image_url):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    image = process_image(image)
    outputs = model(image)
    
    _, preds = torch.max(outputs, 1)
    print(preds)
    return class_names[preds[0]]


In [17]:
# Exemplo de uso
image_path = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSnBk3mcLfRL7JYfLmjEfRuaKzeXVBkdF9GGA&s'
predicted_class = predict(image_path)

print('Produto:', predicted_class)

tensor([121])
Produto: lenço umedecido


In [11]:
class_names

['tatame eva infantil',
 'prendedor chupeta',
 'lixa unha bebe',
 'colchao de berço',
 'andador de bebe',
 'capacete para crianca',
 'joelheira para bebe',
 'nicho',
 'short de bebe',
 'absorvente seio',
 'pacote de algodão',
 'colmeia organizadora de gaveta',
 'lápis de cor',
 'regua crescimento infantil',
 'canguru para bebe',
 'extensor de body',
 'mochila costas',
 'andador  para bebe ',
 'dosador de remedio',
 'sapatenis para bebe',
 'espacador para remedios',
 'capa de chuva infantil',
 'estojo',
 'cadeira de alimentaçao de bebe',
 'pratinhos para bebe',
 'kimono infantil',
 'inalador para bebe',
 'aspirador nasal',
 'concha seio',
 'tinta guache',
 'termometro banheira  para bebe',
 'boliche de brinquedo',
 'fralda  para bebe',
 'termometro',
 'caderno escolar',
 'armazenador para leite materno',
 'gravata  suspensorio',
 'cabide compra',
 'almofada de pescoço',
 'monitor fetal',
 'massinha de modelar',
 'amigurumi',
 'escova de mamadeira',
 'porta lenço  umidecido ',
 'porta le